In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [159]:
# URL of the webpage containing the spell list
base_url = 'https://www.dandwiki.com'
spells_url = f'{base_url}/wiki/5e_SRD:Spells'

# Send a GET request to the webpage
response = requests.get(spells_url)
response.raise_for_status() 

In [160]:
soup = BeautifulSoup(response.content, 'html.parser')

In [173]:
def get_spell_details(spell_url):
    damage_types = ['piercing, bludgeoning, slashing, cold, fire, lightning, thunder, poison, acid, necrotic, radiant, force, psychic']
    response = requests.get(base_url + spell_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    spell_details = {}
    if soup.find('caption'):
        spell_name = soup.find('caption').text.strip()
        spell_details['Name'] = spell_name
    
        labels = soup.find('table', class_='d20').find('td').text.strip().split(' ')
        if '(ritual)' in labels:
            spell_details['Ritual'] = True
        else:
            spell_details['Ritual'] = False

        if 'cantrip' in labels:
            spell_details['Level'] = 'Cantrip'
            spell_details['School'] = labels[0]
        else:
            spell_details['Level'] = labels[0][0]
            spell_details['School'] = labels[1]
            
        spell_details_table = soup.find('table', class_='d20').find_all('tr')
        for row in spell_details_table:
            cells = row.find_all(['th', 'td'])
            if len(cells) == 2:
                key = cells[0].text.strip().rstrip(':')
                value = cells[1].text.strip()
                spell_details[key] = value
        description_paragraphs = soup.find_all('p')
        description = '\n'.join([p.text.strip() for p in description_paragraphs if not p.find('a', title="Main Page")])
    
    
        spell_details['Description'] = description
    
        damage_occurrences = len(re.findall('damage', description))
        init_index = 0
        damage_type_list = []
        for i in range(damage_occurrences):
            index_of_damage = description[init_index:].find('damage')
            if index_of_damage != -1:
                last_space_before_damage = description.rfind(" ", 0, index_of_damage-1)
                damage_type = description[last_space_before_damage+1:index_of_damage-1]
                if damage_type in damage_types:
                    damage_type_list.append(damage_type)
                init_index = index_of_damage
        if damage_type_list:
            spell_details['Damage Type'] = set(damage_type_list)
        else:
            spell_details['Damage Type'] = 'Non-damaging'
            
        index_of_attack = description.find('spell attack')
        if index_of_attack != -1:
            last_space_before_attack = description.rfind(" ", 0, index_of_attack-1)
            attack_type = description[last_space_before_attack+1:index_of_attack-1]
            spell_details['Attack Type'] = attack_type
        else:
            spell_details['Attack Type'] = None
        
        index_of_save = description.find('saving throw')
        if index_of_save != -1:
            last_space_before_save = description.rfind(" ", 0, index_of_save-1)
            save_type = description[last_space_before_save+1:index_of_save-1]
            spell_details['Saving Throw'] = save_type
        else:
            spell_details['Saving Throw'] = None
    
        
        return spell_details

In [175]:
spells_list = []
a_tag_list = []
ul_list = []
for ul in soup.find_all('ul')[5:15]: # find all unordered lists # Need 5 to 14 which are the spells
    ul_list.append (ul)
    for li in ul.find_all('li'): # find all list items per unordered list
        a_tag = li.find('a')
        a_tag_list.append(a_tag)
        if a_tag and 'href' in a_tag.attrs:
            spell_url = a_tag['href'] # this gets the relative url attached to a_tag
            spell_details = get_spell_details(spell_url)
            spells_list.append(spell_details)
            time.sleep(0.5) # be polite to the server

spells_list_filtered = [entry for entry in spells_list if entry is not None]
spells_df = pd.DataFrame(spells_list_filtered)

# Save the DataFrame to a CSV file
spells_df.to_csv('5e_spells.csv', index=False)
# spells_df = pd.DataFrame(spells_list)
#spells_df.to_csv('5e_spells.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'keys'